In [1]:
import pymongo
import pandas as pd

# 连接数据库
client = pymongo.MongoClient('localhost', 27017)
db = client['douban']
table = db['movie_items']

# # 读取数据
# data = pd.DataFrame(list(table.find()))

# # 选择需要显示的字段
# # data = data[['id']]

data=table.find({},{'_id':0,'id':1} )
ids=[i['id'] for i in data]
ids

['26985127',
 '1292001',
 '1849031',
 '25662329',
 '1292213',
 '1929463',
 '1889243',
 '1652587',
 '1292064',
 '3878007',
 '26683290',
 '4920528',
 '26425063',
 '1291549',
 '2131459',
 '2129039',
 '4920389',
 '4739952',
 '1291560',
 '26336252',
 '1292215',
 '26861685',
 '1307914',
 '1299398',
 '20495023',
 '1292370',
 '27110296',
 '26363254',
 '1866479',
 '1293839',
 '1292063',
 '1309046',
 '1306249',
 '26925317',
 '25964071',
 '10574622',
 '24751756',
 '1295124',
 '1292343',
 '1292223',
 '26698897',
 '26862829',
 '4873490',
 '30163509',
 '26366496',
 '25986180',
 '3287562',
 '1485260',
 '25986662',
 '1851857',
 '25815034',
 '25805741',
 '11529526',
 '1978709',
 '10574468',
 '26580232',
 '6786002',
 '1292402',
 '1298070',
 '1907966',
 '1292220',
 '1308807',
 '1302425',
 '26636712',
 '19944106',
 '26325320',
 '27622447',
 '1291841',
 '2334904',
 '1292000',
 '3075287',
 '2209573',
 '11026735',
 '1297447',
 '27133303',
 '25934014',
 '27060077',
 '1291543',
 '1298624',
 '1482072',
 '129158

In [2]:
import random
def get_headers():
    with open('user_agent.txt','r') as f:
        users=f.readlines()
        users=[i.rstrip('\n') for i in users]
    return{'User-Agent':random.choice(users)}
def get_proxies():
    with open('final_douban_https.txt','r') as f:
        https=f.readlines()
        https=[i.rstrip('\n') for i in https]
    return{'https':random.choice(https)}
def get_cookies():
    cookie='bid=hVBCSRNPRjQ; douban-fav-remind=1; gr_user_id=6dd3c308-8536-4566-bce3-4e605c205df8; _vwo_uuid_v2=D2D6CC396CCCA1406E1277D5C515963B2|cf08af69025f901fc58b9d4c33bcfb81; ll="108288"; __yadk_uid=V2tg6WiXtdYv5yAgKCibK1ucFpFU1woR; push_noty_num=0; push_doumail_num=0; __gads=ID=0d6684e69bdb55d4:T=1556710974:S=ALNI_MZM0d22XSAtRv6bwAgIDVH1Uw2F0w; viewed="30471284_25862578_10607365_27180929_30173624_30236842_26977414_30275915"; ct=y; __utmv=30149280.19596; ps=y; __utmc=30149280; __utmc=223695111; __utma=30149280.1352673927.1556942686.1557060678.1557108936.7; __utmz=30149280.1557108936.7.5.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; ap_v=0,6.0; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1557108937%2C%22https%3A%2F%2Fwww.douban.com%2F%22%5D; _pk_ses.100001.4cf6=*; __utma=223695111.969100824.1556547778.1557060678.1557108937.13; __utmb=223695111.0.10.1557108937; __utmz=223695111.1557108937.13.8.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt_douban=1; __utmt=1; __utmb=30149280.5.10.1557108936; _pk_id.100001.4cf6=5d08a517ff50e27f.1556547777.13.1557110472.1557060678.'
    cookie_dict = {i.split("=")[0]:i.split("=")[-1] for i in cookie.split("; ")}
    return cookie_dict
# print(get_headers())
# print(get_proxies())
# print(get_cookies())

In [ ]:
import requests
from lxml import etree
import re
import pymongo
import time
import random
from multiprocessing import Pool
from dns_cache import _setDNSCache

client=pymongo.MongoClient('localhost',27017)#连接数据库
douban=client['douban']#创建新数据库
movie_comments=douban['movie_comments']#创建数据集合
insert_num=0


def get_comment_url(idcard):#获取每部电影短评的链接
    raw_url='https://movie.douban.com/subject/'+idcard+'/comments?start={}&limit=20&sort=new_score&status=P'
    urls=[raw_url.format(x) for x in range(0,220,20)]
    for url in urls:
        get_comment_info(url)
        
def get_comment_info(url):#获取电影详细信息
    print(url)
    _setDNSCache()
    try:
        proxy=get_proxies()
        html=requests.get(url,headers=get_headers(),cookies=get_cookies(),proxies=proxy,timeout=2)
    except:
        print(proxy,'异常')
        return get_comment_info(url)

    response=etree.HTML(html.text)
    if response.xpath('//div[@class="comment-item"]')==[]:
        print(proxy,'可以登录，IP受限')
        return get_comment_info(url)
    
    for item in response.xpath('//div[@class="comment-item"]'):
        comment = {}
        try:
            #电影唯一id
            comment['movie_id'] = response.xpath('//div[@class="fright"]/a/@name')[0].split('-')[-1]
            # 短评的唯一id
            comment['comment_id'] = int(item.xpath('div[@class="comment"]/h3/span[@class="comment-vote"]/input/@value')[0].strip())
            # 多少人评论有用
            comment['useful_num'] = item.xpath('div[@class="comment"]/h3/span[@class="comment-vote"]/span/text()')[0].strip()
            # 评分
            comment['star'] = item.xpath('div[@class="comment"]/h3/span[@class="comment-info"]/span[2]/@class')[0].strip()
            # 评论时间
            comment['time'] = item.xpath('div[@class="comment"]/h3/span[@class="comment-info"]/span[@class="comment-time "]/text()')[0].strip()
            # 评论内容
            comment['content'] = item.xpath('div[@class="comment"]/p/span/text()')[0].strip()
            # 评论者名字（唯一）
            comment['people'] = item.xpath('div[@class="avatar"]/a/@title')[0].strip()
            # 评论者页面
            comment['people_url'] = item.xpath('div[@class="avatar"]/a/@href')[0].strip()
            
            global insert_num
            insert_num+=1
            print('第',insert_num,'次插入数据')
            movie_comments.insert_one(comment)
            print(comment)
        except IndexError:
            print(url,':发生IndexError')
            pass
    
if __name__=='__main__':
    pool=Pool(4)
    for i in ids:
        pool.apply_async(get_comment_url(i))
    pool.close()
    pool.join()

https://movie.douban.com/subject/26985127/comments?start=0&limit=20&sort=new_score&status=P
{'https': '49.86.183.204:9999'} 异常
https://movie.douban.com/subject/26985127/comments?start=0&limit=20&sort=new_score&status=P
第 1 次插入数据
{'movie_id': '26985127', 'comment_id': 1429838363, 'useful_num': '2767', 'star': 'allstar50 rating', 'time': '2018-08-08', 'content': '预告片真的误事儿，是我看过的第一部预告片拖正片后腿的电影。本来看完预告片之后有点担心，但黄渤作为新人导演没有让我失望，结束后的鼓掌发自内心。可以说是我近年来看过的最好的华语电影，超过了药神，细节控会格外开心。晚点来补长评。', 'people': 'rrr', 'people_url': 'https://www.douban.com/people/nasna/', '_id': ObjectId('5ccfabfca1cf182a1cc86c2a')}
第 2 次插入数据
{'movie_id': '26985127', 'comment_id': 1422635404, 'useful_num': '15727', 'star': 'allstar40 rating', 'time': '2018-08-09', 'content': '荒诞中揭示人性，导演处女作应该说太用心了，简直就是想费力而不只是想讨好(巧)观众；场景和特效都非常认真，荒岛，破船的造型……电影就是高度文明的人再退回到猴子原始时代的现代寓言，这也足见一个酝酿多年的演而优则导的处女作的良苦『野心』，如果多一些这样的的导演，那么我们的国产電影好看了', 'people': '与碟私奔', 'people_url': 'https://www.douban.com/people/1305791/', '_id': ObjectId('5ccfabfca1cf182a1cc86c2b')

第 21 次插入数据
{'movie_id': '26985127', 'comment_id': 1430419992, 'useful_num': '2698', 'star': 'allstar40 rating', 'time': '2018-08-08', 'content': '黄渤这一次真的拼了，用一群人荒岛求生的故事外壳好好地搞了一出人性实验，黑色，荒诞，疯狂，从生存的法则，部落的建立，文明的诞生，货币的诞生讲到政权的形成以及乌托邦社会的破灭，而在其中穿插着一个男人犹如摩西一般的成长历程，整个剧本的野心极大，不论是宏观上还是微观上都言之有物，连张艺兴都会演戏了！', 'people': '谢谢你们的鱼', 'people_url': 'https://www.douban.com/people/reave/', '_id': ObjectId('5ccfac00a1cf182a1cc86c3e')}
第 22 次插入数据
{'movie_id': '26985127', 'comment_id': 1431424070, 'useful_num': '1495', 'star': 'allstar10 rating', 'time': '2018-08-10', 'content': '中国不适合拍这种片。', 'people': '库克船长', 'people_url': 'https://www.douban.com/people/162940856/', '_id': ObjectId('5ccfac00a1cf182a1cc86c3f')}
第 23 次插入数据
{'movie_id': '26985127', 'comment_id': 1431130042, 'useful_num': '2363', 'star': 'allstar40 rating', 'time': '2018-08-09', 'content': '简明人类发展史，乌托邦寓言，反向《蝇王》。黄渤导演处女作野心很大，荒诞讽刺的手法反映现实意义，三个阶段代表了生存→物质→精神的进阶，内核表达：文明是人类兽性的遮羞布。而最终的和谐告诉我们，真相才是统治者们最害怕的东西。电影更希望人能直面人性之恶，这样才能认识自身的不完美。新导演总是想表达很多内容，还有政治经济学在里面，

第 61 次插入数据
{'movie_id': '26985127', 'comment_id': 1431364936, 'useful_num': '168', 'star': 'allstar20 rating', 'time': '2018-08-10', 'content': '这不就是《极限挑战》大电影吗？', 'people': '电影黑匣', 'people_url': 'https://www.douban.com/people/171405818/', '_id': ObjectId('5ccfac01a1cf182a1cc86c66')}
第 62 次插入数据
{'movie_id': '26985127', 'comment_id': 1431862549, 'useful_num': '165', 'star': 'allstar20 rating', 'time': '2018-08-10', 'content': '没形成视听风格，因为演技有限，大量无意义对白拉扯空间等原因，镜头碎片化且综艺味太重。它想表达的，古今中外的艺术作品有更深刻的思考，说什么社会寓言，道什么荒诞大戏，不过是给它贴金而已。一堆编剧，也只能保障故事流畅，台词无深刻可能。蜥蜴之眼这种小技，真人秀玩过很多了。不过贵在真诚。', 'people': '南悠一', 'people_url': 'https://www.douban.com/people/Allan.Vr/', '_id': ObjectId('5ccfac01a1cf182a1cc86c67')}
第 63 次插入数据
{'movie_id': '26985127', 'comment_id': 1424716398, 'useful_num': '946', 'star': 'allstar40 rating', 'time': '2018-08-02', 'content': '有幸看了提前点映，两个多小时的长片在现在的国产片里已经很难得了，更特别的是此片的类型完全打破传统，看得出黄渤非常有想法，尝试了很多不同元素，也致敬了一些经典电影，值得鼓励。虽然剧本和人物冲突制造方面仍有待成熟的发展空间，但瑕不掩瑜，还是非常值得一看的。', 'people': '又又', 'people_url': 'https

第 101 次插入数据
{'movie_id': '26985127', 'comment_id': 1431369199, 'useful_num': '60', 'star': 'allstar30 rating', 'time': '2018-08-10', 'content': '差点以为《大闹天竺2.0》呢，看来担心是多余的！事实证明，导演门槛虽低，但黄渤比王宝强会讲故事。', 'people': '江南开爷', 'people_url': 'https://www.douban.com/people/hyk897573/', '_id': ObjectId('5ccfac02a1cf182a1cc86c8e')}
第 102 次插入数据
{'movie_id': '26985127', 'comment_id': 1429809952, 'useful_num': '297', 'star': 'allstar40 rating', 'time': '2018-08-08', 'content': '8月7号上海首映礼mark！是好片没错了！已经准备好要写三篇稿子了哈哈哈等我', 'people': '李多余', 'people_url': 'https://www.douban.com/people/laroky/', '_id': ObjectId('5ccfac02a1cf182a1cc86c8f')}
第 103 次插入数据
{'movie_id': '26985127', 'comment_id': 1431485572, 'useful_num': '105', 'star': 'allstar10 rating', 'time': '2018-08-10', 'content': '电影给及格分，黄渤能拍出来这样的东西真的很不错，但是张艺兴能不能不要再出现在黄渤的电影里了？他好好去演综艺吧，别来祸害电影行业了。', 'people': '傻蛋_最帅', 'people_url': 'https://www.douban.com/people/172304900/', '_id': ObjectId('5ccfac02a1cf182a1cc86c90')}
第 104 次插入数据
{'movie_id': '26985127', 'comme

第 141 次插入数据
{'movie_id': '26985127', 'comment_id': 1429296970, 'useful_num': '231', 'star': 'allstar40 rating', 'time': '2018-08-07', 'content': '初涉导演的黄渤，却有着极大的野心，其想在自己的处女座《一出好戏》所容纳的是一场关乎人类社会演变历程的再现，而电影以荒岛求生的展开，在求生中衍生以逗人捧腹的荒诞，在荒诞中用人性博弈所建立的社会，为这一看似晦涩的命题，寻觅到了一个颇接地气的落足。', 'people': '梦里诗书', 'people_url': 'https://www.douban.com/people/57542378/', '_id': ObjectId('5ccfac03a1cf182a1cc86cb6')}
第 142 次插入数据
{'movie_id': '26985127', 'comment_id': 1427808308, 'useful_num': '144', 'star': 'allstar40 rating', 'time': '2018-08-06', 'content': '想象着黄渤首执导筒的处女作的种种面貌，但万万没想到是这样的，如此特别，又如此别扭。荒诞、悲伤掺杂着点黑色幽默。但电影语言中的缺点与不足显而易见。七个编剧，其中一个是张冀。插曲王菲唱的，片尾曲窦靖童唱的。结尾俩彩蛋有宁浩、徐峥、管虎等咖导客串。', 'people': '无人应答', 'people_url': 'https://www.douban.com/people/2349644/', '_id': ObjectId('5ccfac03a1cf182a1cc86cb7')}
第 143 次插入数据
{'movie_id': '26985127', 'comment_id': 1438041290, 'useful_num': '15', 'star': 'allstar50 rating', 'time': '2018-08-15', 'content': '之前黄渤确实拍过好作品，但是后面模式化的出演也很多……嗯，这部真是让人久违的眼前一亮，充满魔幻色彩，让我们看到了中国电影的更多可能，蛮好的。支持黄渤，拒

{'https': '49.86.183.204:9999'} 异常
https://movie.douban.com/subject/26985127/comments?start=180&limit=20&sort=new_score&status=P
{'https': '123.163.96.194:9999'} 可以登录，IP受限
https://movie.douban.com/subject/26985127/comments?start=180&limit=20&sort=new_score&status=P
第 181 次插入数据
{'movie_id': '26985127', 'comment_id': 1433660768, 'useful_num': '13', 'star': 'allstar30 rating', 'time': '2018-08-12', 'content': '3.5。看预告一副烂片相，竟然还不错。黄导的电影还是对得起他的演技的，虽然戏剧转变上略嫌简陋，但看得出制作用心。王迅王宝强套路演法，舒淇是令人出戏的花瓶，黄导最用心调教竟是张艺兴，都算有情有义。', 'people': '五色全味', 'people_url': 'https://www.douban.com/people/wusequanwei/', '_id': ObjectId('5ccfac09a1cf182a1cc86cde')}
第 182 次插入数据
{'movie_id': '26985127', 'comment_id': 1434705072, 'useful_num': '19', 'star': 'allstar20 rating', 'time': '2018-08-12', 'content': '如果是王宝强导的我可以接受。', 'people': '黑狗成', 'people_url': 'https://www.douban.com/people/44457357/', '_id': ObjectId('5ccfac09a1cf182a1cc86cdf')}
第 183 次插入数据
{'movie_id': '26985127', 'comment_id': 1432276887, 'useful_num': '10', 's

{'https': '119.130.105.120:3128'} 异常
https://movie.douban.com/subject/1292001/comments?start=0&limit=20&sort=new_score&status=P
{'https': '119.130.105.120:3128'} 异常
https://movie.douban.com/subject/1292001/comments?start=0&limit=20&sort=new_score&status=P
{'https': '49.86.183.204:9999'} 异常
https://movie.douban.com/subject/1292001/comments?start=0&limit=20&sort=new_score&status=P
第 221 次插入数据
{'movie_id': '1292001', 'comment_id': 12062592, 'useful_num': '669', 'star': 'allstar20 rating', 'time': '2007-11-08', 'content': '也就音乐还算可以，Tim Roth太猥琐，不适合演这种文绉绉的角色', 'people': '大宸', 'people_url': 'https://www.douban.com/people/bladerunner2/', '_id': ObjectId('5ccfac13a1cf182a1cc86d06')}
第 222 次插入数据
{'movie_id': '1292001', 'comment_id': 11636999, 'useful_num': '4832', 'star': 'allstar40 rating', 'time': '2007-03-12', 'content': '我总觉得这是一个真实的故事，因为他的懦弱，懦弱得真实。', 'people': 'Soblue', 'people_url': 'https://www.douban.com/people/soblue.solo/', '_id': ObjectId('5ccfac13a1cf182a1cc86d07')}
第 223 次插入数据
{'movi

{'https': '123.163.96.194:9999'} 可以登录，IP受限
https://movie.douban.com/subject/1292001/comments?start=40&limit=20&sort=new_score&status=P
第 261 次插入数据
{'movie_id': '1292001', 'comment_id': 1897756, 'useful_num': '2428', 'star': 'allstar10 rating', 'time': '2006-03-12', 'content': '侮辱钢琴，侮辱“好”电影。', 'people': '荔枝超人', 'people_url': 'https://www.douban.com/people/richer725/', '_id': ObjectId('5ccfac1ca1cf182a1cc86d2e')}
第 262 次插入数据
{'movie_id': '1292001', 'comment_id': 896761278, 'useful_num': '49', 'star': 'allstar50 rating', 'time': '2015-02-24', 'content': '很多年前看的时候怎么都不懂1900为什么不下船，今天下午重新再看了一遍，居然特别感同身受。其实，我也从来没有下过船。', 'people': '夏小暖', 'people_url': 'https://www.douban.com/people/candybaby/', '_id': ObjectId('5ccfac1ca1cf182a1cc86d2f')}
第 263 次插入数据
{'movie_id': '1292001', 'comment_id': 431411306, 'useful_num': '27', 'star': 'allstar50 rating', 'time': '2011-08-30', 'content': '《海上钢琴师》：一生一世，献给音乐和海洋！——曾经，我以电影和音乐在我心中的地位来表示《天堂电影院》比《海》的最爱悬殊；如今，两次资料馆大屏幕上的重温，虽然都比不上第一遍的观感，但让我明白我确实喜爱《天堂电影院》超过《海上》一点；我的眼